In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp fastai.data.load

In [4]:
# export
# Python native modules
import os
from typing import Callable
# Third party libs
from fastcore.all import *
import torchdata.datapipes as dp
from torch.utils.data.dataloader_experimental import DataLoader2
from torch.utils.data.graph import traverse
# Local modules
from fastrl.fastai.data.loop.core import *

from fastrl.fastai.data.pipes.map.mux import *
from fastrl.fastai.data.pipes.map.demux import *

# Loading
> Objects using the `Loop` and `DataPipe` API for DataLoading

We will replicate [fastai mnist loading](https://colab.research.google.com/github/fastai/fastbook/blob/master/04_mnist_basics.ipynb).

In [9]:
untar_data??

Signature:
untar_data(
    url,
    archive=None,
    data=None,
    c_key='data',
    force_download=False,
)
Source:   
def untar_data(url, archive=None, data=None, c_key='data', force_download=False):#, extract_func=file_extract, timeout=4):
    "Download `url` to `fname` if `dest` doesn't exist, and extract to folder `dest`"
    d = FastDownload(fastai_cfg(), module=fastai.data, archive=archive, data=data, base='~/.fastai')
    return d.get(url, force=force_download, extract_key=c_key)
File:      /opt/conda/lib/python3.7/site-packages/fastai/data/external.py
Type:      function


In [13]:
get_image_files??

Signature: get_image_files(path, recurse=True, folders=None)
Source:   
def get_image_files(path, recurse=True, folders=None):
    "Get image files in `path` recursively, only in `folders`, if specified."
    return get_files(path, extensions=image_extensions, recurse=recurse, folders=folders)
File:      /opt/conda/lib/python3.7/site-packages/fastai/data/transforms.py
Type:      function


In [12]:
from fastai.vision.all import get_image_files,PILImage,ToTensor,PILBase
from fastrl.fastai.data.external import untar_data,URLs,get_image_files
from fastrl.fastai.torch_core import *
path = untar_data(URLs.MNIST_SAMPLE)

(path/'train').ls()

ImportError: cannot import name 'get_image_files' from 'fastrl.fastai.data.external' (/home/fastrl_user/fastrl/fastrl/fastai/data/external.py)

First we create the dataset...

In [ ]:
# export
class TypeTransformLoop(dp.map.MapDataPipe):
    def __init__(self,datapipe, type_tfms):
        self.type_tfms,self.datapipe = Pipeline(type_tfms),datapipe
    
    @callback_getitem
    def __getitem__(self, index):
        data = self.datapipe[index]
        return self.type_tfms(data)
            
    def __len__(self): return len(self.datapipe)
    
class ItemTransformLoop(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe, item_tfms:List[Callable]): 
        self.item_tfms,self.source_datapipe = Pipeline(item_tfms),source_datapipe
    @callback_iter
    def __iter__(self):
        for data in self.source_datapipe:
            yield self.item_tfms(data)
    
    
class BatchTransformLoop(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe, batch_tfms):
        self.batch_tfms,self.source_datapipe = Pipeline(batch_tfms),source_datapipe
    @callback_iter
    def __iter__(self):
        for data in self.source_datapipe:
            yield self.batch_tfms(data)

In [ ]:
# export
def default_loader_loop(
    items:Iterable,
    splitter:Callable,
    cbs:Optional[List[Callback]]=None,
    type_tfms:Optional[Transform]=None,
    item_tfms:Optional[Transform]=None,
    batch_tfms:Optional[Transform]=None,
    bs:int=2,
    shuffler:Optional[Union[dp.iter.IterDataPipe,dp.map.MapDataPipe]]=None
):
    type_tfms = ifnone(type_tfms,L())
    pipe = dp.map.SequenceWrapper(items)
    train_dp,valid_dp = DemultiplexerMapDataPipe(
        pipe,
        num_instances=2,
        classifier_fn=splitter,
        drop_none=True
    )
    train_dp,valid_dp = L(train_dp,valid_dp).map(TypeTransformLoop, type_tfms=type_tfms)
    if shuffler:
        train_dp,valid_dp = L(train_dp,valid_dp).map(shuffler)
    else:
        train_dp,valid_dp = L(train_dp,valid_dp).map(Self.shuffle())
    train_dp,valid_dp = L(train_dp,valid_dp).map(dp.iter.MapToIterConverter)
    train_dp,valid_dp = L(train_dp,valid_dp).map(ItemTransformLoop, item_tfms=ifnone(item_tfms,L()))
    train_dp,valid_dp = train_dp.batch(bs),valid_dp.batch(bs)
    return train_dp,valid_dp

In [ ]:
# export
def GrandparentSplitter(train='train',valid='valid'):
    def splitter(item):
        if all(s not in item.parts for s in (train,valid)): return None
        if item.is_dir(): return None
        # valid=1, train=0
        return valid in item.parts 
    return splitter

In [ ]:
base = default_loader_loop(
    L(path.rglob('*')),
    GrandparentSplitter(),
    type_tfms = L(PILImage.create,ToTensor)
)

In [ ]:

default_constructor(
    dp.iter.Zipper(*base),   
    cbs=L()
)

In [ ]:
train_dl,valid_dl = DataLoader2(base[0]),DataLoader2(base[1])

In [ ]:
for i,element in enumerate(train_dl):
    print(element)
    break

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script(silent=True)